In [2]:
import pandas as pd
import json
from collections import defaultdict
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
df = pd.read_csv('../data/api-info.csv')
df.db_name = df.db_name.fillna('')
df.report_name = df.report_name.fillna('')
df.main_view = df.main_view.fillna('')
df.table_1 = df.table_1.fillna('')
df.table_2 = df.table_2.fillna('')
df.table_3 = df.table_3.fillna('')
# df.tab_impact = df.tab_impact.fillna('')
df.lookback = df.lookback.fillna(0)
# df.lookback = df.lookback.replace('year-to-date', 100)
df

,source,source_name,db_name,report_name,lookback,main_view,table_1,table_2,table_3
0,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Standard,14,[dbo].[vw_StandardReport],,,
1,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Standard,14,[dbo].[FactStandardReport],,,
2,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Activity,30,[dbo].[vw_Floodlight],,,
3,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Activity,30,[dbo].[FactFloodlight],,,
4,Ad Server / Creative / Planned Spend Sources,Campaign Manager,DCM_UAT,Monthly Reach,365,[dbo].[vw_CookieRNF_Month],,,
...,...,...,...,...,...,...,...,...,...
317,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[foursquare].[dim_advertiser],,,
318,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[foursquare].[fact_reports],[foursquare].[dim_reports],,
319,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[dbo].[vw_FourSquare_Overall_Master],,,
320,TV / Placed / DAR Sources,Foursquare,ApiReportingDB,Report name,0,[foursquare].[dim_advertiser],,,


In [19]:
for v1, v2 in zip(df.report_name.values, df.lookback.values):
    print(v1, v2)

Standard 14
Standard 14
Activity  30
Activity  30
Monthly Reach 365
Monthly Reach 365
Weekly Reach 365
Weekly Reach 365
DMA 7
DMA 7
Standard 14
Standard 14
Activity 30
Activity 30
Monthly Reach 365
Weekly Reach 365
Search Standard 0
Search Standard 0
Search Activity 0
Search Activity 0
DCO Standard 7
DCO Standard 7
Ad Details 0
Ad Details 0
Ad Details 0
Ad Details 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Prisma Data 0
Prisma Data 0
Prisma Data 0
Prisma Data 0
Advanced Placement Details 0
Placements Daily 0
Buy Orders 0
IO Monthly 0
Report name 14
Report name 14
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Standard 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Floodlight 0
Report name 7
Report name 7
Custom 7
Custom 7
Viewability 7
Viewability 7
Monitoring 7
Monitoring

array([ 14,  14,  30,  30, 365, 365, 365, 365,   7,   7,  14,  14,  30,
        30, 365, 365,   0,   0,   0,   0,   7,   7,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        14,  14,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   7,   7,   7,   7,   7,   7,   7,   7,   7,   0,
         0,   0,   0,   0,   0,   0,   0,   0,  14,  14,  14,  14,  14,
         0,   0,   0,   0,   0,   0,   0,   7,   7,   7,   7,   7,   7,
         7,   7,   7,   7,   7,   7,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,
         7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,
         7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,  14,
        14,  14,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [13]:
def find_idx_or_add_new_child(L, key, value):
    for i, line in enumerate(L):
        if line[key] == value:
            return i
    L.append({key: value, 'name': value, 'children': []})
    return len(L)-1


nested = []
for index, row in df.iterrows():
    expanding = nested
    for col in df.columns:
        if col == 'lookback': continue #skipping lookback since it has been already added at the previous level
        if 'table' in col and row[col] == '': break
            
        idx = find_idx_or_add_new_child(expanding, col, row[col])
        if col == 'report_name':
            expanding[idx]['lookback'] = row['lookback'] #adding lookback to the same level as report name
        expanding = expanding[idx]['children']

In [15]:
nested[0]['children'][0]['children'][0]['children'][0].keys()

dict_keys(['report_name', 'name', 'children', 'lookback'])

In [20]:
json.dump(nested, open('../data/nested_no_lookback.json', 'w'))